In [1]:
from pyspark.sql import SparkSession
from  pyspark.sql.functions import *
import random

# Criando uma SparkSession
spark = SparkSession.builder.appName("Exemplo").getOrCreate()

# Gerando dados simulados para clientes
num_clientes = 1000
clientes_data = [
    (i, f"Cliente {i}", f"Cidade {random.randint(1, 10)}", random.choice(["Masculino", "Feminino"]), random.randint(18, 80))
    for i in range(1, num_clientes + 1)
]

# Criando o DataFrame de clientes
df_clientes = spark.createDataFrame(clientes_data, ["cliente_id", "nome", "cidade", "genero", "idade"])

# Gerando dados simulados para creditos
num_creditos = 5000
creditos_data = [
    (i, random.randint(1, num_creditos), random.uniform(100, 5000), random.choice(["A", "B", "C"]), random.randint(1, 12))
    for i in range(1, num_creditos + 1)
]

# Criando o DataFrame de creditos
df_creditos = spark.createDataFrame(creditos_data, ["credito_id", "cliente_id", "valor_credito", "tipo_credito", "mes"])

# Gerando dados simulados para contas
df_contas = spark.createDataFrame([
    (i, random.randint(1, num_clientes), random.uniform(1000, 10000))
    for i in range(1, num_clientes + 1)
], ["conta_id", "cliente_id", "saldo"])

# Gerando dados simulados para transacoes
df_transacoes = spark.createDataFrame([
    (i, random.randint(1, num_clientes), random.randint(1, num_clientes))
    for i in range(1, num_creditos + 1)
], ["transacao_id", "conta_id", "cliente_id"])

# Gerando dados simulados para pagamentos
df_pagamentos = spark.createDataFrame([
    (i, random.randint(1, num_creditos), random.uniform(10, 500))
    for i in range(1, num_creditos + 1)
], ["pagamento_id", "transacao_id", "valor_pago"])


23/12/24 15:09:19 WARN Utils: Your hostname, DESKTOP-0KANSSR resolves to a loopback address: 127.0.1.1; using 192.168.189.146 instead (on interface eth0)
23/12/24 15:09:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/12/24 15:09:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_clientes.createOrReplaceTempView("clientes")
df_creditos.createOrReplaceTempView("creditos")
df_contas.createOrReplaceTempView("contas")
df_transacoes.createOrReplaceTempView("transacoes")
df_pagamentos.createOrReplaceTempView("pagamentos")

In [3]:
resultado = spark.sql("""
WITH credit_summary AS (
    SELECT
        c.cliente_id,
        SUM(cr.valor_credito) AS total_credito,
        COUNT(cr.credito_id) AS numero_creditos
    FROM
        creditos cr
    JOIN clientes c ON cr.cliente_id = c.cliente_id
    GROUP BY
        c.cliente_id
    HAVING
        SUM(cr.valor_credito) > 10000
)

SELECT
    cl.nome,
    ct.conta_id,
    ct.saldo,
    cs.total_credito,
    cs.numero_creditos,
    CASE
        WHEN ct.saldo - cs.total_credito < 0 THEN 'Saldo Negativo'
        ELSE 'Saldo Positivo'
    END AS status_conta,
    SUM(p.valor_pago) AS total_pago
FROM
    clientes cl
JOIN contas ct ON cl.cliente_id = ct.cliente_id
JOIN credit_summary cs ON cl.cliente_id = cs.cliente_id
JOIN transacoes tr ON ct.conta_id = tr.conta_id
JOIN pagamentos p ON tr.transacao_id = p.transacao_id
--WHERE
   -- cl.cliente_id IN (SELECT cliente_id FROM credit_summary)
GROUP BY
    cl.nome, ct.conta_id, ct.saldo, cs.total_credito, cs.numero_creditos
ORDER BY SUM(p.valor_pago) DESC
""")

# Exibindo o resultado da consulta
resultado.show(3)

+-----------+--------+------------------+------------------+---------------+--------------+------------------+
|       nome|conta_id|             saldo|     total_credito|numero_creditos|  status_conta|        total_pago|
+-----------+--------+------------------+------------------+---------------+--------------+------------------+
|Cliente 591|     945|1141.1321801350769|13843.261165130922|              4|Saldo Negativo|  3634.96563095157|
|Cliente 946|     592|1630.7692544808922|10804.808439787457|              3|Saldo Negativo|1617.0459924678482|
|Cliente 515|     699|  7296.05379699182| 10028.38311129087|              3|Saldo Negativo| 1488.883575572318|
+-----------+--------+------------------+------------------+---------------+--------------+------------------+
only showing top 3 rows



In [4]:
df_credit_summary = df_creditos.alias("cr")\
    .join(df_clientes.alias("c"), "cliente_id", "inner")\
    .groupBy("cliente_id")\
    .agg(sum("valor_credito").alias("total_credito"), count("credito_id").alias("numero_creditos"))\
    .filter("total_credito > 10000")

In [5]:
df_clientes.alias("cl")\
    .join(df_contas.alias("ct"), "cliente_id")\
    .join(df_credit_summary.alias("cs"), "cliente_id")\
    .join(df_transacoes.alias("tr"), "conta_id")\
    .join(df_pagamentos.alias("p"), "transacao_id")\
    .groupBy("cl.nome", "ct.conta_id", "ct.saldo", "cs.total_credito", "cs.numero_creditos")\
    .agg(sum("p.valor_pago").alias("total_pago"))\
    .select(
        "nome",
        "conta_id",
        "saldo",
        "total_credito",
        "numero_creditos",
        when(
            col("saldo") - col("total_credito") < 0, "Saldo Negativo"
        ).otherwise("Saldo Positivo").alias("status_conta"),
        "total_pago"
    )\
    .orderBy(col("total_pago").desc())\
    .show(3)

+-----------+--------+------------------+------------------+---------------+--------------+------------------+
|       nome|conta_id|             saldo|     total_credito|numero_creditos|  status_conta|        total_pago|
+-----------+--------+------------------+------------------+---------------+--------------+------------------+
|Cliente 591|     945|1141.1321801350769|13843.261165130922|              4|Saldo Negativo|  3634.96563095157|
|Cliente 946|     592|1630.7692544808922|10804.808439787457|              3|Saldo Negativo|1617.0459924678482|
|Cliente 515|     699|  7296.05379699182| 10028.38311129087|              3|Saldo Negativo| 1488.883575572318|
+-----------+--------+------------------+------------------+---------------+--------------+------------------+
only showing top 3 rows

